## Objetivo

Abrir o link https://pbdatatrader.com.br/jogosdodia e extrair a tabela de jogos para o python

### Processo

* Abrir a página e esperar tudo ser carregado nela
* Dar scroll na tabela e guardar em um dataframe

In [2]:
import time
from selenium import webdriver
import pandas as pd

##ABRE O SITE E ESPERA CARREGAR=================
driver = webdriver.Chrome()

link= "https://pbdatatrader.com.br/jogosdodia"
driver.get(link)

#Tabela está dentro de um iframe que está dentro de outro iframe-------------------
#Para esperar a página carregar completa, vamo pegar um elemento da própria tabela

while len(driver.find_elements_by_tag_name("iframe")) == 0:#espera carregar o primeiro iframe
    time.sleep(1)

first_iframe = driver.find_elements_by_tag_name("iframe")[0]
driver.switch_to.frame(first_iframe) #muda o foco para o primeiro iframe


while len(driver.find_elements_by_tag_name("iframe")) == 0:#espera carregar o segundo iframe
    time.sleep(1)

second_iframe = driver.find_elements_by_tag_name("iframe")[0]
driver.switch_to.frame(second_iframe) #muda o foco para o segundo iframe

data = driver.find_element_by_xpath('//*[@id="pvExplorationHost"]/div/div/exploration/div/explore-canvas/div/div[2]/div/div[2]/div[2]/visual-container-repeat/visual-container[19]/transform/div/div[3]/div/visual-modern/div/div/div[2]/div[1]/div[2]/div/div[1]/div')
print("Carregou a tabela toda") 

Carregou a tabela toda


Para identificar o scroll da tabela interna, rodei esse script no console da página
```JavaScript
var elements = document.getElementsByTagName("div");

for (var i = 0; i < elements.length; i++) { 
    if (elements[i].scrollHeight > document.getElementsByTagName("body")[0].scrollHeight) {
        console.log(elements[i]);
        console.log(elements[i].scrollHeight);
    }
}
```

E localizei qual o elemento que tinha a tabela.

Fica dando scroll na tabela e absorvendo os dados

In [4]:
classe = "bodyCells" #Elemento da tabela
tabela_df = pd.DataFrame #cria tabela vazia

numero_de_scroll = 50

for i in range(numero_de_scroll):
    #Dá scroll na tabela
    posicao_scroll = i * 250
    driver.execute_script(f"document.getElementsByClassName('bodyCells')[0].scroll(0,{posicao_scroll})")#desce a tabela
    
    #lê os textos da tabela e guarda em listas
    tabela = driver.find_element_by_class_name("innerContainer") #guarda todos os textos em ordem separados por \n
    lista_texto = tabela.text.split("\n")#cria tabela de textos a partir do texto carregado por separado \n
    lista_colunas = lista_texto[:12] #separa linhas e colunas
    lista_valores  =lista_texto[12:] #separa linhas e colunas
    
    #cria um dicionário com os elementos carregados
    dic = {}
    for coluna in lista_colunas:
        dic[coluna] = []
        for j in range(20):
            dic[coluna].append(lista_valores[j])
        lista_valores = lista_valores[20:]
    
    #transforma o dicionário em um dataframe
    if tabela_df.empty:
        tabela_df = pd.DataFrame.from_dict(dic)
        print('criou tabela')
    else:
        tabela_df = tabela_df.append(pd.DataFrame.from_dict(dic), ignore_index = True)
    
#Após carregar tudo terão vários dados repetidos porque o scroll foi de pouco em pouco
tabela_df = tabela_df.drop_duplicates()
display(tabela_df)

criou tabela


,Data,Hora,País,Campeonato,Mandante,Visitante,Pontos por Jogo Mandante,Pontos por Jogo Visitante,Média de Gols do Confronto,% BTTS,% Over 0.5 FT,% Over 1.5 FT
0,02/01/2022,13:30,Senegal,Senegal Premier League,Pikine,Casa Sport,"1,33","2,00","1,42",42,03/01/2022,10:30
1,05/01/2022,10:00,Zambia,Super League,Kafue Celtic,Prison Leopards,"1,38","1,19","1,63",25,03/01/2022,08:30
2,02/01/2022,10:00,Algeria,Ligue 1,RC Arbaâ,CS Constantine,"0,60","1,89","1,68",35,03/01/2022,10:30
3,02/01/2022,11:00,Algeria,Ligue 1,JS Kabylie,MC Oran,"2,00","0,80","1,45",23,03/01/2022,17:00
4,02/01/2022,12:00,Nigeria,NPFL,Wikki Tourist,MFM,"3,00","1,00","1,50",0,03/01/2022,08:30
...,...,...,...,...,...,...,...,...,...,...,...,...
555,03/01/2022,12:00,England,Counties Leagues Eastern Counties League,Haverhill Rovers,Thetford Town FC,"0,86","1,05","3,39",64,100,87
556,03/01/2022,12:00,England,Counties Leagues Eastern Counties League,Lakenheath,Newmarket Town FC,"1,10","1,60","3,60",68,96,88
557,03/01/2022,12:00,England,Counties Leagues Eastern Counties League,Mulbarton Wanderers FC,Swaffham Town,"2,10","0,33","4,04",57,100,95
558,03/01/2022,12:00,England,Counties Leagues Eastern Counties League,Walsham Le Willows,Hadleigh United,"1,74","1,05","3,15",53,95,79
